# W&B End-to-End Workshop: Aquatic Species Classification (AQUA20)

## From First Baseline to Production-Ready Model with Full MLOps

---

### Workshop Story

**Scenario:** You're part of a marine biology AI research team building an image classifier to identify aquatic species from underwater photographs. The goal is to help marine researchers automatically catalog and monitor marine biodiversity.

**Your journey:**
1. **Train a baseline model** with proper experiment tracking and visual diagnostics
2. **Package the dataset and model** as versioned artifacts with rich metadata
3. **Register the model** and promote it through stages (candidate → staging → production)
4. **Create CI/CD-style automation** for evaluation and promotion workflows

---

### What You'll Learn

| Section | Topic | Key Skills |
|---------|-------|------------|
| 1 | Setup | Environment configuration, W&B login, config objects |
| 2 | Data & Artifacts | HuggingFace datasets, **raw artifact upload**, **split artifacts with lineage** |
| 3 | Experiment Tracking | Run initialization, logging profiles, step alignment, resume |
| 4 | Model Training | PyTorch training, mixed precision, checkpoint logging |
| 5 | Visual Logging | Images, tables, confusion matrices, PR curves |
| 6 | Model Artifacts | Model artifact with lineage, use_artifact consumer workflow |
| 7 | Registry | Collections, linking, promotion workflow |
| 8 | CI/CD Automation | Validation jobs, gating, programmatic promotion |
| 9 | Offline Mode | WANDB_MODE, syncing runs |
| 10 | Wrap-up | Recap, next steps |

**Key Change:** Artifacts are introduced in Section 2 - the very first thing you do after loading data is log it as an artifact!

---

### Prerequisites

- Basic Python and PyTorch knowledge
- Google Colab with T4 GPU runtime
- W&B account (free at wandb.ai)

**Runtime:** ~30-45 minutes with fast_run=True, ~60-90 minutes with full training

---

## 1. Setup

Let's install dependencies, authenticate with W&B, and configure our experiment.

In [ ]:
# Install required packages
!pip install wandb datasets transformers timm -qU

In [ ]:
# Imports
import random
import datetime
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.cuda.amp import GradScaler
import wandb
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
from datetime import datetime
import json
import os

# Workshop utilities (handles ML boilerplate)
from workshop_utils import (
    CLASS_NAMES, NUM_CLASSES, DEVICE,
    set_seed, get_transforms,
    create_model, count_parameters,
    train_one_epoch, evaluate,
    generate_run_name, update_best_metric,
    AquaticDataset,
    # Visualization helpers
    create_prediction_images, create_predictions_table,
)

print(f"PyTorch: {torch.__version__}")
print(f"Device: {DEVICE}")
print(f"W&B: {wandb.__version__}")


In [ ]:
# ============================================================================
# W&B CONFIGURATION - UPDATE THESE FOR YOUR ENVIRONMENT
# ============================================================================

WANDB_ENTITY = "m-bakir"              # Your W&B username or team
WANDB_PROJECT = "SIE-Workshop-2026"    # Project name
# WANDB_ORG = "Moe"                     # Organization (for registry)

# Authenticate with W&B
WANDB_HOST = "https://mbakir.wandb.io/" #@param
wandb.login(host= WANDB_HOST)

In [ ]:
# ============================================================================
# MASTER CONFIGURATION
# ============================================================================
# Single config object for all hyperparameters and run metadata
# This is the "source of truth" for reproducibility

# ============================================================================
# WORKSHOP: Random Model Assignment
# ============================================================================
# ============================================================================
# TRAINING CONFIGURATION
# ============================================================================
# Random model assignment - creates diverse runs for W&B comparison!
WORKSHOP_MODELS = ["resnet50", "efficientnet_b0"]
ASSIGNED_MODEL = random.choice(WORKSHOP_MODELS)
print(f"Your assigned model: {ASSIGNED_MODEL}")

# Training config - these get logged to W&B automatically
CONFIG = {
    "model_name": ASSIGNED_MODEL,
    "num_classes": NUM_CLASSES,
    "epochs": 3,              # Quick training for workshop
    "batch_size": 32,
    "learning_rate": 1e-3,
    "weight_decay": 1e-4,
    "image_size": 224,
    "max_samples": 1000,      # Subset for fast iteration
    "use_amp": True,
    "seed": 42,
}

# Set reproducibility
set_seed(CONFIG["seed"])
print(f"\nConfig: {CONFIG}")

# Device is already set by workshop_utils
print(f"Using device: {DEVICE}")

---

## 2. Data Preparation

The AQUA20 dataset has been **pre-prepared and logged as W&B artifacts**. This is a common pattern in production ML:
- Data engineering teams prepare and version datasets
- ML engineers consume versioned datasets for training

**Available artifacts:**
- `aqua-train:v0` - Training split (~6,500 images)
- `aqua-val:v0` - Validation split (~800 images)  
- `aqua-test:v0` - Test split (~800 images)

The `workshop_utils.py` file handles all ML boilerplate (transforms, dataset class, etc.).

**Key W&B concept:** The artifact download happens in the training run using `use_artifact()` - this creates **lineage** showing exactly which dataset version was used.

In [ ]:
# ============================================================================
# W&B ARTIFACT PATHS (Pre-prepared datasets)
# ============================================================================
ARTIFACT_PROJECT = f"{WANDB_ENTITY}/{WANDB_PROJECT}"
TRAIN_ARTIFACT = f"{ARTIFACT_PROJECT}/aqua-train:v0"
VAL_ARTIFACT = f"{ARTIFACT_PROJECT}/aqua-val:v0"
TEST_ARTIFACT = f"{ARTIFACT_PROJECT}/aqua-test:v0"

print(f"Artifacts: {TRAIN_ARTIFACT}, {VAL_ARTIFACT}, {TEST_ARTIFACT}")
print(f"Classes: {NUM_CLASSES} marine species")

In [ ]:
# Transforms are provided by workshop_utils.get_transforms()
# Includes underwater-optimized augmentations: color jitter, rotation, etc.
print(f"Image transforms ready (size: {CONFIG['image_size']}x{CONFIG['image_size']})")

In [ ]:
# AquaticDataset is provided by workshop_utils
# Loads images from class folders in downloaded artifacts
print("AquaticDataset class ready")

In [ ]:
# Section 2 complete - transforms and Dataset class are ready
# 
# The actual data loading happens in the TRAINING RUN (Section 4):
# 1. wandb.init() starts the training run
# 2. use_artifact() downloads train + val datasets (creates lineage!)
# 3. Datasets and DataLoaders are created
# 4. Training proceeds with metrics logged to W&B
#
# This pattern ensures your training run shows exactly which 
# dataset versions were used - critical for reproducibility!

print("✅ Section 2 complete!")
print("   - Transforms defined")
print("   - AquaticDataset class ready")
print("   - Data will be downloaded in training run via use_artifact()")

In [ ]:
# ============================================================================
# DATA EXPLORATION (Pre-prepared in W&B)
# ============================================================================
# An EDA table has been prepared for you in the W&B project!
# 
# Go to: https://wandb.ai/m-bakir/SIE-Workshop-2026
# Look for the "dataset-eda-exploration" run
#
# The table includes:
# - Sample images from each class
# - Image statistics (brightness, contrast, color channels)
# - Class distribution visualization
#
# Try these explorations:
# 1. Group by "class" to see samples per species
# 2. Sort by "brightness" to find dark/light images
# 3. Filter by "blue_ratio" to see underwater color cast
# 4. Compare "contrast" across different marine species

print("📊 Data exploration table available in W&B!")
print("   Project: m-bakir/SIE-Workshop-2026")
print("   Run: dataset-eda-exploration")
print("\n   Skip to Section 4 to start training!")

### Why This Matters: Data Sanity Checks TODO Remove?

Before training any model, always verify:
1. **Images load correctly** - No corrupted files
2. **Labels are valid** - Within expected range
3. **Class distribution** - Check for imbalance (common in marine datasets)
4. **Visual inspection** - Do underwater images look right?

Logging this to W&B creates a permanent record of your data exploration.

---

## 3. Experiment Tracking Fundamentals

Now let's set up proper experiment tracking with W&B best practices.

In [ ]:
# generate_run_name() creates descriptive names from config
# Format: {model}-lr{learning_rate}-bs{batch_size}-ep{epochs}
print(f"Example run name: {generate_run_name(CONFIG)}")

In [ ]:
from typing import Dict

# Best Practice 3: Step Alignment
def log_training_step(metrics: Dict[str, float], step: int, run):
    """Log training metrics with proper step alignment."""
    wandb.log(metrics, step=step)

# Best Practice 4: Summary vs History
def update_summary(run, key: str, value: float, mode: str = "max"):
    """Update run summary with best value."""
    current_best = run.summary.get(key)

    if current_best is None:
        run.summary[key] = value
    elif mode == "max" and value > current_best:
        run.summary[key] = value
    elif mode == "min" and value < current_best:
        run.summary[key] = value

print("✅ Experiment tracking utilities defined")

---

## 4. Model Training

Train a baseline model with proper experiment tracking.

**W&B Features in this section:**
- `tags` and `group` - Organize runs for filtering and comparison
- `notes` - Quick description visible in run overview  
- `define_metric()` - Set epoch as x-axis for cleaner charts
- `use_artifact()` - Download data with automatic lineage tracking
- `wandb.alert()` - Get notified when validation improves
- **System Metrics** - GPU/CPU usage logged automatically (check System tab in W&B UI!)


In [ ]:
# Create model (uses timm library under the hood)
model = create_model(CONFIG["model_name"], NUM_CLASSES, pretrained=True)
model = model.to(DEVICE)

total_params, trainable_params = count_parameters(model)
print(f"Model: {CONFIG['model_name']}")
print(f"  Parameters: {total_params:,} ({trainable_params:,} trainable)")

In [ ]:
# train_one_epoch() and evaluate() are imported from workshop_utils
# They handle mixed precision training and W&B logging automatically
print("Training utilities ready")

In [ ]:
# ============================================================================
# MAIN TRAINING LOOP
# ============================================================================

run_name = generate_run_name(CONFIG)

run = wandb.init(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    name=run_name,
    job_type="training",
    # GROUP: Links related runs together (all resnet50 runs grouped)
    group=CONFIG["model_name"],
    # TAGS: Filterable labels - find runs by model, dataset, experiment type
    tags=[
        "aqua20",                      # Dataset
        "baseline",                    # Experiment type
        CONFIG["model_name"],          # Model architecture
        "workshop-uk-2026",            # Workshop identifier
        f"epochs-{CONFIG['epochs']}",  # Hyperparameter tag
    ],
    # NOTES: Quick description (visible in run overview)
    notes=f"Workshop training: {CONFIG['model_name']} on AQUA20. "
          f"Epochs: {CONFIG['epochs']}, LR: {CONFIG['learning_rate']}, BS: {CONFIG['batch_size']}",
    config=CONFIG
)

# DEFINE_METRIC: Set "epoch" as x-axis for cleaner charts
wandb.define_metric("epoch")
wandb.define_metric("train/*", step_metric="epoch")
wandb.define_metric("val/*", step_metric="epoch")
wandb.define_metric("learning_rate", step_metric="epoch")

print(f"\nStarting training run: {run_name}")
print(f"   View at: {run.url}")
print(f"   Tags: {run.tags}")

# ============================================================================
# DOWNLOAD DATASET ARTIFACTS (Creates Lineage!)
# ============================================================================
# Using use_artifact() within this run creates lineage showing this training
# depends on these specific dataset versions - critical for reproducibility!

print("\n📦 Downloading dataset artifacts...")

# Download training data
train_artifact = run.use_artifact(TRAIN_ARTIFACT, type='dataset')
train_dir = train_artifact.download()
print(f"   ✅ Training data: {train_artifact.name}:{train_artifact.version}")

# Download validation data
val_artifact = run.use_artifact(VAL_ARTIFACT, type='dataset')
val_dir = val_artifact.download()
print(f"   ✅ Validation data: {val_artifact.name}:{val_artifact.version}")

# ============================================================================
# CREATE DATASETS AND DATALOADERS
# ============================================================================
print("\n📊 Creating datasets...")

train_dataset = AquaticDataset(
    train_dir, 
    transform=get_transforms(CONFIG["image_size"], is_training=True), 
    class_names=CLASS_NAMES,
    max_samples=CONFIG.get("max_samples")  # Limit for fast_run mode
)

val_dataset = AquaticDataset(
    val_dir,
    transform=get_transforms(CONFIG["image_size"], is_training=False),
    class_names=CLASS_NAMES
)

train_loader = DataLoader(
    train_dataset,
    batch_size=CONFIG["batch_size"],
    shuffle=True,
    num_workers=0,
    pin_memory=True,
    drop_last=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=CONFIG["batch_size"],
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

print(f"   Train: {len(train_dataset)} samples, {len(train_loader)} batches")
print(f"   Val: {len(val_dataset)} samples, {len(val_loader)} batches")

# Download test data for final evaluation
test_artifact = run.use_artifact(TEST_ARTIFACT, type='dataset')
test_dir = test_artifact.download()
print(f"   Test data: {test_artifact.name}:{test_artifact.version}")

test_dataset = AquaticDataset(
    test_dir,
    transform=get_transforms(CONFIG["image_size"], is_training=False),
    class_names=CLASS_NAMES
)

test_loader = DataLoader(
    test_dataset,
    batch_size=CONFIG["batch_size"],
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

print(f"   Test: {len(test_dataset)} samples, {len(test_loader)} batches")

# ============================================================================
# SETUP TRAINING COMPONENTS
# ============================================================================
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(
    model.parameters(),
    lr=CONFIG["learning_rate"],
    weight_decay=CONFIG["weight_decay"]
)
scheduler = optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=CONFIG["epochs"]
)
scaler = GradScaler(enabled=CONFIG["use_amp"])

best_val_acc = 0.0
best_model_path = None

for epoch in range(CONFIG["epochs"]):
    print(f"\n{'='*60}")
    print(f"Epoch {epoch+1}/{CONFIG['epochs']}")
    print(f"{'='*60}")

    train_loss, train_acc = train_one_epoch(
        model, train_loader, criterion, optimizer, scaler, DEVICE,
        epoch, log_interval=1, run=run  # Log every batch
    )

    val_loss, val_acc, val_preds, val_labels, val_probs = evaluate(
        model, val_loader, criterion, DEVICE, desc=f"Epoch {epoch+1} [Val]"
    )

    scheduler.step()
    current_lr = scheduler.get_last_lr()[0]

    epoch_metrics = {
        "epoch": epoch + 1,
        "train/loss": train_loss,
        "train/accuracy": train_acc,
        "val/loss": val_loss,
        "val/accuracy": val_acc,
        "learning_rate": current_lr,
    }
    wandb.log(epoch_metrics)

    print(f"\nEpoch {epoch+1} Results:")
    print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
    print(f"  Learning Rate: {current_lr:.6f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_path = f"best_model_epoch{epoch+1}.pth"
        torch.save({
            "epoch": epoch + 1,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "val_acc": val_acc,
            "val_loss": val_loss,
            "config": CONFIG
        }, best_model_path)
        print(f"  New best model saved! Val Acc: {val_acc:.2f}%")

        # ALERT: Send notification when new best model is found
        wandb.alert(
            title="New Best Model!",
            text=f"Validation accuracy improved to {val_acc:.2f}% at epoch {epoch+1}",
            level=wandb.AlertLevel.INFO
        )

        run.summary["best_val_accuracy"] = val_acc
        run.summary["best_val_loss"] = val_loss
        run.summary["best_epoch"] = epoch + 1

    # ========================================================================
    # LOG MODEL CHECKPOINT AS ARTIFACT (creates versioned history)
    # ========================================================================
    checkpoint_artifact = wandb.Artifact(
        name=f"model-{CONFIG['model_name']}",
        type="model",
        metadata={
            "epoch": epoch + 1,
            "val_accuracy": val_acc,
            "val_loss": val_loss,
            "train_accuracy": train_acc,
            "train_loss": train_loss,
        }
    )
    
    # Save checkpoint to temp file
    checkpoint_path = f"checkpoint_epoch{epoch+1}.pth"
    torch.save({
        "epoch": epoch + 1,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "config": CONFIG
    }, checkpoint_path)
    checkpoint_artifact.add_file(checkpoint_path)
    
    # Aliases for easy reference
    aliases = [f"epoch_{epoch+1}"]
    if val_acc >= best_val_acc:
        aliases.append("best")
    if epoch == CONFIG["epochs"] - 1:
        aliases.append("latest")
    
    run.log_artifact(checkpoint_artifact, aliases=aliases)
    print(f"  Logged checkpoint artifact (aliases: {aliases})")

print(f"\n")
print(f"Training complete!")
print(f"Best validation accuracy: {best_val_acc:.2f}%")

---

## 5. Visual Logging (Media)

Log rich visual diagnostics: underwater images, predictions, confusion matrices, and more.

In [ ]:
# Final evaluation on test set
print("\nEvaluating on test set...")

checkpoint = torch.load(best_model_path)
model.load_state_dict(checkpoint["model_state_dict"])

test_loss, test_acc, test_preds, test_labels, test_probs = evaluate(
    model, test_loader, criterion, DEVICE, desc="Test Evaluation"
)

print(f"\nTest Results:")
print(f"  Test Loss: {test_loss:.4f}")
print(f"  Test Accuracy: {test_acc:.2f}%")

wandb.log({
    "test/loss": test_loss,
    "test/accuracy": test_acc
})

run.summary["test_accuracy"] = test_acc
run.summary["test_loss"] = test_loss

In [ ]:
# Log prediction samples with images and confidence scores
# create_prediction_images() handles the boilerplate

prediction_images = create_prediction_images(
    test_dataset, test_preds, test_probs, CLASS_NAMES, n_samples=16
)

# Log to W&B - images appear in Media tab
wandb.log({"predictions/samples": prediction_images})

print(f"Logged {len(prediction_images)} prediction samples to W&B")


In [ ]:
# Create W&B Table with predictions for detailed analysis
# Includes per-class confidence scores for histogram visualization

predictions_table = create_predictions_table(
    test_dataset, test_preds, test_probs, CLASS_NAMES, n_samples=100
)

# Log to W&B - table appears in Tables tab
wandb.log({"predictions/analysis_table": predictions_table})

print(f"Logged predictions table with {len(CLASS_NAMES)} class score columns")
print("  In W&B UI try:")
print("  - Group by 'truth' to see recall per class")
print("  - Group by 'guess' to see precision per class")
print("  - Filter: row['truth'] != row['guess'] to find errors")

In [ ]:
# Log per-class metrics
# Get unique classes that appear in the data (handles fast_run with subset)
unique_classes = sorted(set(test_labels) | set(test_preds))

precision, recall, f1, support = precision_recall_fscore_support(
    test_labels, test_preds, labels=unique_classes, average=None, zero_division=0
)

metrics_table = wandb.Table(
    columns=["Class", "Precision", "Recall", "F1-Score", "Support"]
)

for i, class_idx in enumerate(unique_classes):
    class_name = CLASS_NAMES[class_idx] if class_idx < len(CLASS_NAMES) else f"Class_{class_idx}"
    metrics_table.add_data(
        class_name,
        round(precision[i], 4),
        round(recall[i], 4),
        round(f1[i], 4),
        int(support[i])
    )

wandb.log({"evaluation/per_class_metrics": metrics_table})

precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
    test_labels, test_preds, average='macro'
)

wandb.log({
    "evaluation/precision_macro": precision_macro,
    "evaluation/recall_macro": recall_macro,
    "evaluation/f1_macro": f1_macro
})

run.summary["precision_macro"] = precision_macro
run.summary["recall_macro"] = recall_macro
run.summary["f1_macro"] = f1_macro

print("\nPer-class metrics logged.")
print(f"Macro F1-Score: {f1_macro:.4f}")

---

## 6. Model Artifacts with Lineage

We've already logged our **data artifacts** in Section 2:
- ✅ Raw dataset artifact (`aqua20-raw`)
- ✅ Split artifacts with lineage (`aqua-train`, `aqua-val`, `aqua-test`)

Now it's time to complete the lineage by **logging the trained model as an artifact** that references the training data!

### Complete Lineage Chain

```
Raw Dataset → Train/Val/Test Splits → Model
```

By using `use_artifact()` to consume the split artifacts when logging our model, we create a complete audit trail showing exactly which data was used to train this model.

In [ ]:
# ============================================================================
# MODEL ARTIFACT - Complete the Lineage Chain
# ============================================================================
# Start a run that consumes the split artifacts and logs the trained model.
# This creates the final lineage: Splits → Model

model_artifact_run = wandb.init(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    name="aqua20-model-artifact-logging",
    job_type="model-logging",
    tags=["aqua20", "model-artifact", "baseline"],
    notes="Log trained model artifact with lineage to training data splits"
)

# Reference the SAME artifacts used during training (creates proper lineage)
train_artifact_ref = model_artifact_run.use_artifact(TRAIN_ARTIFACT, type='dataset')
val_artifact_ref = model_artifact_run.use_artifact(VAL_ARTIFACT, type='dataset')

print(f"📦 Model trained using:")
print(f"   Train artifact: {train_artifact_ref.name}:{train_artifact_ref.version}")
print(f"   Val artifact: {val_artifact_ref.name}:{val_artifact_ref.version}")

# Prepare model info
model_info = {
    "architecture": CONFIG["model_name"],
    "num_classes": NUM_CLASSES,
    "class_names": CLASS_NAMES,
    "input_size": CONFIG["image_size"],
    "pretrained": True,
    "total_params": total_params,
    "trainable_params": trainable_params,
    "training_config": {
        "epochs": CONFIG["epochs"],
        "batch_size": CONFIG["batch_size"],
        "learning_rate": CONFIG["learning_rate"],
        "optimizer": "AdamW"
    },
    "metrics": {
        "best_val_accuracy": best_val_acc,
        "test_accuracy": test_acc,
        "test_loss": test_loss,
        "f1_macro": f1_macro
    },
    "data_artifacts": {
        "train": f"{train_artifact_ref.name}:{train_artifact_ref.version}",
        "val": f"{val_artifact_ref.name}:{val_artifact_ref.version}"
    },
    "created_at": datetime.now().isoformat(),
    "training_run_id": model_artifact_run.id
}

with open("artifacts/model_info.json", "w") as f:
    json.dump(model_info, f, indent=2)

import shutil
shutil.copy(best_model_path, "artifacts/model.pth")

# Create and log the model artifact
model_artifact = wandb.Artifact(
    name="aqua20-species-classifier",
    type="model",
    description=f"Aquatic Species classifier ({CONFIG['model_name']}) trained on AQUA20 dataset",
    metadata={
        "architecture": CONFIG["model_name"],
        "num_classes": NUM_CLASSES,
        "input_size": CONFIG["image_size"],
        "final_val_accuracy": best_val_acc,
        "final_test_accuracy": test_acc,
        "f1_macro": f1_macro,
        "framework": "pytorch",
        "domain": "marine_biology",
        "train_artifact": f"{train_artifact_ref.name}:{train_artifact_ref.version}",
        "val_artifact": f"{val_artifact_ref.name}:{val_artifact_ref.version}"
    }
)

model_artifact.add_file("artifacts/model.pth")
model_artifact.add_file("artifacts/model_info.json")

model_artifact_run.log_artifact(model_artifact, aliases=["latest", "candidate", "baseline"], tags=["baseline-model", "aqua20", "marine-biology"])

print("\n✅ Model artifact logged with lineage!")
print(f"   Name: aqua20-species-classifier")
print(f"   Type: model")
print(f"   Test Accuracy: {test_acc:.2f}%")
print(f"   Aliases: latest, candidate, baseline")

wandb.finish()

In [ ]:
# ============================================================================
# COMPLETE ARTIFACT LINEAGE SUMMARY
# ============================================================================
# At this point, we've created the following artifact lineage:
#
# Section 2 (Data):
#   1. Raw Dataset Artifact (aqua20-raw)
#      ↓ use_artifact() in data-splitting run
#   2. Split Artifacts (aqua-train, aqua-val, aqua-test)
#
# Section 6 (Model Artifacts):
#      ↓ use_artifact() in model-artifact-logging run
#   3. Model Artifact (aqua20-species-classifier)
#
# Check the W&B UI → Artifacts → Lineage tab to visualize this!

print("="*60)
print("COMPLETE ARTIFACT LINEAGE")
print("="*60)
print("""
                    ┌──────────────────────┐
                    │  aqua20-raw  │
                    │   (source-of-truth)  │
                    └──────────┬───────────┘
                               │
              ┌────────────────┼────────────────┐
              ▼                ▼                ▼
       ┌────────────┐  ┌────────────┐  ┌────────────┐
       │ train-split│  │ val-split  │  │ test-split │
       └──────┬─────┘  └──────┬─────┘  └────────────┘
              │               │
              └───────┬───────┘
                      ▼
          ┌─────────────────────┐
          │ aqua20-species-     │
          │ classifier (model)  │
          └─────────────────────┘
""")
print("✅ View lineage at: W&B UI → Artifacts → Select any artifact → Lineage tab")
print("="*60)

In [ ]:
# ============================================================================
# DEMONSTRATE use_artifact: LOAD MODEL FROM ARTIFACT
# ============================================================================
# This shows how another run can consume your model artifact.
# This is useful for evaluation, deployment, or further fine-tuning.

eval_run = wandb.init(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    name="aqua20-model-evaluation-from-artifact",
    job_type="evaluation",
    tags=["aqua20", "evaluation", "artifact-consumption"],
    notes="Evaluation job that loads model from artifact"
)

# Load the model artifact we just created
artifact_path = f"{WANDB_ENTITY}/{WANDB_PROJECT}/aqua20-species-classifier:candidate"
loaded_model_artifact = eval_run.use_artifact(artifact_path, type="model")

print(f"\n📦 Using artifact: {artifact_path}")
print(f"   Version: {loaded_model_artifact.version}")
print(f"   Metadata: {loaded_model_artifact.metadata}")

# Download the artifact files
artifact_dir = loaded_model_artifact.download()
print(f"   Downloaded to: {artifact_dir}")

# Load the model from the artifact
model_path = os.path.join(artifact_dir, "model.pth")
checkpoint = torch.load(model_path, map_location=DEVICE)

eval_model = create_model(
    CONFIG["model_name"],
    CONFIG["num_classes"],
    pretrained=False
)
eval_model.load_state_dict(checkpoint["model_state_dict"])
eval_model = eval_model.to(DEVICE)
eval_model.eval()

print("\n✅ Model loaded from artifact successfully!")

# Quick evaluation
eval_loss, eval_acc, _, _, _ = evaluate(
    eval_model, test_loader, criterion, DEVICE, desc="Artifact Model Eval"
)

wandb.log({
    "evaluation/loss": eval_loss,
    "evaluation/accuracy": eval_acc,
    "artifact_version": loaded_model_artifact.version
})

print(f"\nEvaluation from artifact:")
print(f"  Accuracy: {eval_acc:.2f}%")
print(f"  Loss: {eval_loss:.4f}")

wandb.finish()
print("\n✅ Evaluation run finished")

In [ ]:
# ============================================================================
# KEY ARTIFACT CONCEPTS RECAP
# ============================================================================

print("""
╔══════════════════════════════════════════════════════════════════╗
║                    ARTIFACT KEY CONCEPTS                         ║
╠══════════════════════════════════════════════════════════════════╣
║                                                                  ║
║  1. PRODUCER WORKFLOW (log_artifact)                             ║
║     - Create artifact with wandb.Artifact()                      ║
║     - Add files with artifact.add_file()                         ║
║     - Log with run.log_artifact()                                ║
║                                                                  ║
║  2. CONSUMER WORKFLOW (use_artifact)                             ║
║     - Fetch with run.use_artifact("entity/project/name:alias")   ║
║     - Download with artifact.download()                          ║
║     - Creates INPUT lineage automatically!                       ║
║                                                                  ║
║  3. LINEAGE                                                      ║
║     - use_artifact() → creates INPUT edge                        ║
║     - log_artifact() → creates OUTPUT edge                       ║
║     - Automatically tracked in W&B UI                            ║
║                                                                  ║
║  4. ALIASES vs VERSIONS                                          ║
║     - Aliases are mutable pointers (latest, production)          ║
║     - Versions are immutable (v0, v1, v2...)                     ║
║                                                                  ║
╚══════════════════════════════════════════════════════════════════╝
""")

---

## 7. Model Registry: Promote to Production

The **Model Registry** is where you promote validated models for deployment.

**Key concepts:**
- **Artifact aliases** = training state (epoch_1, best, latest)
- **Registry aliases** = deployment state (staging, production)

**Workflow:**
1. Training creates model artifacts with checkpoints
2. Link the best artifact to a **Registered Model** (collection)
3. Promote within registry: staging → production


In [ ]:
# ============================================================================
# STEP 1: LINK BEST MODEL TO REGISTRY
# ============================================================================
# Take the best checkpoint from training and link it to a Registered Model

registry_run = wandb.init(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    name="registry-promotion",
    job_type="registry-promotion",
    tags=["registry", "promotion"],
)

# Get the best model artifact from training
best_model_path = f"{WANDB_ENTITY}/{WANDB_PROJECT}/model-{CONFIG['model_name']}:best"
best_artifact = registry_run.use_artifact(best_model_path, type="model")

print(f"Best model artifact: {best_artifact.name}:{best_artifact.version}")
print(f"  Metadata: {best_artifact.metadata}")

# Link to a Registered Model collection in the Registry
# This creates a new collection if it doesn't exist
REGISTRY_NAME = "aqua-classifier"  # Collection name in Registry

registry_run.link_artifact(
    artifact=best_artifact,
    target_path=f"wandb-registry-model/{REGISTRY_NAME}",
    aliases=["staging"]  # Start in staging
)

print(f"\nLinked to Registry: {REGISTRY_NAME}")
print(f"  Alias: staging")
print(f"  Check Model Registry in W&B UI!")


In [ ]:
# ============================================================================
# STEP 2: PROMOTE TO PRODUCTION
# ============================================================================
# After validating in staging, promote to production

# Use the API to add 'production' alias
api = wandb.Api()

# Get the staged model from registry
registry_path = f"{WANDB_ENTITY}/wandb-registry-model/{REGISTRY_NAME}:staging"

try:
    staged_artifact = api.artifact(registry_path)
    
    # Validation check before production
    val_acc = staged_artifact.metadata.get("val_accuracy", 0)
    MIN_ACC_FOR_PRODUCTION = 50.0
    
    print(f"Staged model: {staged_artifact.name}")
    print(f"  Validation accuracy: {val_acc:.2f}%")
    print(f"  Min required: {MIN_ACC_FOR_PRODUCTION}%")
    
    if val_acc >= MIN_ACC_FOR_PRODUCTION:
        # Add production alias
        staged_artifact.aliases.append("production")
        staged_artifact.save()
        
        print(f"\n  PROMOTED TO PRODUCTION!")
        print(f"  Aliases: {staged_artifact.aliases}")
    else:
        print(f"\n  FAILED: Accuracy below threshold")
        
except wandb.errors.CommError as e:
    print(f"Error: Could not find staged model. Run Step 1 first.")
    print(f"  {e}")

wandb.finish()

print("\n" + "="*60)
print("Registry workflow complete!")
print("Check Model Registry in W&B UI to see:")
print("  - aqua-classifier collection")
print("  - staging and production aliases")
print("="*60)


---

## 9. Online vs Offline Runs and Syncing

Understand different run modes and how to sync offline runs.

In [ ]:
# ============================================================================
# ONLINE vs OFFLINE MODE
# ============================================================================

print("""
W&B Run Modes:

1. ONLINE (default)
   - Metrics streamed to W&B servers in real-time
   - Run visible in UI immediately
   - Requires network connection

2. OFFLINE
   - All data saved locally to ./wandb/ directory
   - No network requests during training
   - Must sync later to view in UI
   - Use when: no internet, restricted networks, HPC clusters

3. DISABLED
   - No tracking at all
   - Use for quick local testing

How to set mode:
  - In code: wandb.init(mode="offline")
  - Environment variable: export WANDB_MODE=offline
""")

In [ ]:
# Demonstrate offline mode
print("Creating an OFFLINE run...")

offline_run = wandb.init(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    name="aqua20-offline-demo-run",
    mode="offline",
    job_type="offline-demo",
    tags=["offline", "demo", "aqua20"]
)

print(f"Run directory: {offline_run.dir}")
print(f"Run mode: {offline_run.mode}")

for i in range(10):
    wandb.log({"offline_metric": i * 2, "step": i})

offline_run_dir = offline_run.dir
wandb.finish()

print(f"\n✅ Offline run completed")
print(f"   Data saved to: {offline_run_dir}")

In [ ]:
print("""
To sync offline runs to W&B:

1. Sync a specific run (point to the .wandb file):
   wandb sync wandb/offline-run-YYYYMMDD_HHMMSS-runid/run-runid.wandb

2. Sync all offline runs in a directory:
   wandb sync --sync-all wandb/

3. Check what needs syncing:
   wandb sync --view
""")

# Show the actual sync command for this run
import glob
if 'offline_run_dir' in dir():
    wandb_files = glob.glob(f'{offline_run_dir}/*.wandb')
    if wandb_files:
        print(f"To sync this demo run:")
        print(f"  wandb sync {wandb_files[0]}")


---

## 10. Wrap-up

Let's recap what we built and discuss next steps.

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════════╗
║            AQUA20 WORKSHOP COMPLETE! 🐠🦈🐋                      ║
╚══════════════════════════════════════════════════════════════════╝

What we built:
──────────────
1. ✅ Data Pipeline
   - Loaded AQUA20 Aquatic Species dataset from HuggingFace
   - Created train/val/test splits
   - Logged data sanity checks to W&B

2. ✅ Experiment Tracking
   - Used proper run naming conventions
   - Configured logging profiles
   - Maintained step alignment
   - Tracked summary vs history metrics

3. ✅ Model Training
   - Trained EfficientNet-B0 with mixed precision
   - Logged training curves and checkpoints
   - Saved best model based on validation accuracy

4. ✅ Visual Logging
   - Logged underwater species prediction samples
   - Created W&B Tables for analysis
   - Generated confusion matrix
   - Logged per-class metrics

5. ✅ Artifacts
   - Created dataset artifact with metadata
   - Created model artifact with aliases
   - Demonstrated use_artifact workflow

6. ✅ Registry & Promotion
   - Promoted model: candidate → staging → production
   - Added gated promotion with quality checks
   - Created audit trail with timestamps

7. ✅ CI/CD Automation
   - Implemented training job (Step A)
   - Implemented evaluation job (Step B)
   - Implemented promotion job (Step C)
   - Ran full automated pipeline

8. ✅ Offline Mode
   - Created offline run
   - Learned how to sync runs


9. ✅ Post-Training Analysis (Public API)
   - Queried runs programmatically
   - Compared model architectures
   - Updated run summaries

Domain-specific considerations for marine biology:
──────────────────────────────────────────────────
- Underwater images often have color shifts (blue/green)
- Consider underwater-specific augmentations
- Class imbalance is common (rare species)
- Image quality varies with depth and conditions

Next steps to explore:
──────────────────────
- Try different model architectures (ResNet, ViT)
- Add underwater-specific augmentations
- Implement W&B Sweeps for hyperparameter optimization
- Set up GitHub Actions with the CI/CD scripts
- Explore W&B Reports for sharing results
""")

In [ ]:
# Cleanup temporary files
import shutil

cleanup_files = [
    "artifacts",
    "ci_model_placeholder.json"
]

for f in cleanup_files:
    if os.path.isdir(f):
        shutil.rmtree(f)
        print(f"Removed directory: {f}")
    elif os.path.isfile(f):
        os.remove(f)
        print(f"Removed file: {f}")

for f in os.listdir("."):
    if f.startswith("best_model_") and f.endswith(".pth"):
        os.remove(f)
        print(f"Removed: {f}")

print("\n✅ Cleanup complete!")

---

## 9. Post-Training Analysis with Public API

After multiple workshop participants have trained their models, use the W&B Public API to:
- Fetch run data programmatically
- Compare runs across the project
- Update run summaries after the fact

**Note:** This section works best after several participants have logged runs!


In [ ]:
# Fetch all workshop training runs using the Public API
api = wandb.Api()

# Query runs with our workshop tag
runs = api.runs(
    path=f"{WANDB_ENTITY}/{WANDB_PROJECT}",
    filters={"tags": "workshop-uk-2026"},  # Filter by our workshop tag!
    order="-created_at"  # Most recent first
)

print(f"Found {len(runs)} workshop runs:\n")
for run in runs[:10]:  # Show first 10
    val_acc = run.summary.get('best_val_accuracy', 'N/A')
    model = run.config.get('model_name', 'unknown')
    print(f"  {run.name}: {model} | Val Acc: {val_acc}")
    print(f"    Tags: {run.tags}")
    print(f"    URL: {run.url}\n")


In [ ]:
# Compare model architectures: resnet50 vs efficientnet_b0
import pandas as pd

# Collect data from all workshop runs
run_data = []
for run in runs:
    if run.state == "finished":  # Only completed runs
        run_data.append({
            "name": run.name,
            "model": run.config.get("model_name", "unknown"),
            "learning_rate": run.config.get("learning_rate", None),
            "batch_size": run.config.get("batch_size", None),
            "epochs": run.config.get("epochs", None),
            "best_val_acc": run.summary.get("best_val_accuracy", None),
            "test_acc": run.summary.get("test_accuracy", None),
        })

df = pd.DataFrame(run_data)
print("Workshop Runs Comparison:\n")
display(df)

# Group by model architecture
if len(df) > 0 and 'model' in df.columns:
    print("\nAverage Performance by Model:")
    print(df.groupby('model')['best_val_acc'].agg(['mean', 'std', 'count']))


In [ ]:
# Update a run's summary after the fact
# This is useful for adding computed metrics or notes

# Get a specific run by ID (replace with your run ID)
# You can find run IDs in the W&B URL: wandb.ai/entity/project/runs/RUN_ID

# Example: Add a custom tag to the best performing run
if len(df) > 0:
    # Find the best run
    best_idx = df['best_val_acc'].idxmax()
    best_run_name = df.loc[best_idx, 'name']
    
    # Fetch that run
    for run in runs:
        if run.name == best_run_name:
            print(f"Best run: {run.name}")
            print(f"  Current summary: {dict(run.summary)}")
            
            # Add a custom metric to the summary
            run.summary['workshop_rank'] = 1
            run.summary['notes'] = 'Best performing model in workshop'
            run.summary.update()  # Save changes
            
            print(f"\n  Updated summary!")
            break


In [ ]:
# Access detailed training history (all logged steps)
# Useful for custom visualizations or analysis

if len(runs) > 0:
    # Get the most recent run's history
    recent_run = runs[0]
    history = recent_run.history()
    
    print(f"Run: {recent_run.name}")
    print(f"History shape: {history.shape}")
    print(f"\nAvailable metrics: {list(history.columns)[:10]}...")
    
    # Show epoch-level metrics
    epoch_cols = [c for c in history.columns if 'epoch' in c.lower() or 'train/' in c or 'val/' in c]
    if epoch_cols:
        print(f"\nEpoch-level data:")
        display(history[epoch_cols].dropna().head(10))


---

## Resources

- [W&B Documentation](https://docs.wandb.ai/)
- [W&B Artifacts Guide](https://docs.wandb.ai/guides/artifacts)
- [W&B Model Registry](https://docs.wandb.ai/guides/model-registry)
- [W&B API Reference](https://docs.wandb.ai/ref/python)
- [HuggingFace Datasets](https://huggingface.co/datasets)
- [timm Model Zoo](https://github.com/huggingface/pytorch-image-models)

---

**Thank you for completing this workshop!**